# Hospital Admission Data Analysis

## Project Goals and Objectives:
### 1) Create an effective ETL pipeline
* Utilize Python and relevant packages (pandas, numpy, re) to extract and transform data
* Build a Google Devian 11 Instance
* Perform pipeline creation with Mage VM
* Load onto Google Cloud Storage
### 2) Build an easy to digest dashboard to showcase information
* Use Looker Studio to build a visualization which gives clear insights into data presented
### 3) Advise business decisions based on data presented
* Use Google BigQuery to create logical conclusions based on data presented

In [1]:
import pandas as pd
import re as re

## Load dataset into a DataFrame
df = pd.read_csv('healthcare_dataset.csv')

print(f'There are {len(df[df.duplicated()])} duplicate rows in this dataset. There are also {df.isnull().sum().sum()} null values in the dataset.')

There are 0 duplicate rows in this dataset. There are also 0 null values in the dataset.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                10000 non-null  object 
 1   Age                 10000 non-null  int64  
 2   Gender              10000 non-null  object 
 3   Blood Type          10000 non-null  object 
 4   Medical Condition   10000 non-null  object 
 5   Date of Admission   10000 non-null  object 
 6   Doctor              10000 non-null  object 
 7   Hospital            10000 non-null  object 
 8   Insurance Provider  10000 non-null  object 
 9   Billing Amount      10000 non-null  float64
 10  Room Number         10000 non-null  int64  
 11  Admission Type      10000 non-null  object 
 12  Discharge Date      10000 non-null  object 
 13  Medication          10000 non-null  object 
 14  Test Results        10000 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 1.1+

### From our data, we see that we do not have a suitable primary key which is simple, meaningless, and unique.

In [3]:
## Create a suitable primary key for each admission
df['adm_id'] = df.index

### Our data includes dates which are not of correct type 'datetime'. We will convert these columns, as well as add a new column 'los' with this information which will represent length of stay in days. 

In [4]:
## Convert relevant columns to type datetime
df['Date of Admission'] = pd.to_datetime(df['Date of Admission'])
df['Discharge Date'] = pd.to_datetime(df['Discharge Date'])

## Create a length of stay column
df['los'] = df['Discharge Date']-df['Date of Admission']
df.los.head()

0   14 days
1   14 days
2   30 days
3    1 days
4   24 days
Name: los, dtype: timedelta64[ns]

### When the length of stay column is created, it is recorded as 'x days'. We will convert this to an integer.

In [5]:
## Convert the length of stay column to string type
df['los'] = df['los'].astype(str)

## Remove ' day' or ' days' from each record in length of stay
def remove_days(word):
    return re.sub(r'\s\b\D+\b',"",word)

df['los'] = df['los'].apply(remove_days)
df['los'] = df['los'].astype(int)

df.los.head()

0    14
1    14
2    30
3     1
4    24
Name: los, dtype: int64

### We will rename our columns to be consistent and SQL friendly

In [6]:
df.rename(columns={'Name':'name',
        'Age':'age',
        'Gender':'gender',
        'Blood Type':'blood_type',
        'Medical Condition':'med_condition',
        'Date of Admission':'adm_date',
        'Doctor':'doctor',
        'Hospital':'hospital',
        'Insurance Provider':'ins_provider',
        'Billing Amount':'billing_amt',
        'Room Number':'room_number',
        'Admission Type':'adm_type',
        'Discharge Date':'dc_date',
        'Medication':'medication',
        'Test Results':'test_results'}, inplace=True)

### Let's round the billing amount to two decimal places.

In [7]:
df['billing_amt'] = round(df['billing_amt'],2)
df.head()

,name,age,gender,blood_type,med_condition,adm_date,doctor,hospital,ins_provider,billing_amt,room_number,adm_type,dc_date,medication,test_results,adm_id,los
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.98,146,Elective,2022-12-01,Aspirin,Inconclusive,0,14
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.06,404,Emergency,2023-06-15,Lipitor,Normal,1,14
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.90,292,Emergency,2019-02-08,Lipitor,Normal,2,30
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.32,480,Urgent,2020-05-03,Penicillin,Abnormal,3,1
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.34,477,Urgent,2021-08-02,Paracetamol,Normal,4,24


### We will now create a few dimension tables to assist in data analysis and visualization later.

In [8]:
person_dim = df[['name','age','gender','blood_type','ins_provider']].drop_duplicates().reset_index(drop=True)
person_dim['person_id'] = person_dim.index
person_dim = person_dim[['person_id','name','age','gender','blood_type','ins_provider']]
person_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   person_id     10000 non-null  int64 
 1   name          10000 non-null  object
 2   age           10000 non-null  int64 
 3   gender        10000 non-null  object
 4   blood_type    10000 non-null  object
 5   ins_provider  10000 non-null  object
dtypes: int64(2), object(4)
memory usage: 468.9+ KB


In [9]:
medication_dim = df[['medication']].drop_duplicates().reset_index(drop=True)
medication_dim['medication_id'] = medication_dim.index
medication_dim = medication_dim[['medication_id','medication']]

med_condition_dim = df[['med_condition']].drop_duplicates().reset_index(drop=True)
med_condition_dim['med_condition_id'] = med_condition_dim.index
med_condition_dim = med_condition_dim[['med_condition_id','med_condition']]

### Dividing out our date columns will be helpful to answer questions related to date later.

In [10]:
adm_date_dim = df[['adm_date']].drop_duplicates().reset_index(drop=True)
adm_date_dim['adm_date_year'] = adm_date_dim['adm_date'].dt.year
adm_date_dim['adm_date_month'] = adm_date_dim['adm_date'].dt.month
adm_date_dim['adm_date_day'] = adm_date_dim['adm_date'].dt.day
adm_date_dim['adm_date_weekday'] = adm_date_dim['adm_date'].dt.weekday
adm_date_dim['adm_date_id'] = adm_date_dim.index

dc_date_dim = df[['dc_date']].drop_duplicates().reset_index(drop=True)
dc_date_dim['dc_date_year'] = dc_date_dim['dc_date'].dt.year
dc_date_dim['dc_date_month'] = dc_date_dim['dc_date'].dt.month
dc_date_dim['dc_date_day'] = dc_date_dim['dc_date'].dt.day
dc_date_dim['dc_date_weekday'] = dc_date_dim['dc_date'].dt.weekday
dc_date_dim['dc_date_id'] = dc_date_dim.index

### Let's create our final fact table.

In [11]:
fact_table = df.merge(person_dim, on=['name','gender','age','blood_type','ins_provider']) \
             .merge(adm_date_dim, on='adm_date') \
             .merge(dc_date_dim, on='dc_date') \
             .merge(med_condition_dim, on='med_condition') \
             .merge(medication_dim, on='medication') \
             [['adm_id','person_id',
               'adm_date_id','dc_date_id','med_condition_id','medication_id','los','doctor',
               'hospital','billing_amt','room_number','test_results']]


fact_table.drop_duplicates().reset_index
fact_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   adm_id            10000 non-null  int64  
 1   person_id         10000 non-null  int64  
 2   adm_date_id       10000 non-null  int64  
 3   dc_date_id        10000 non-null  int64  
 4   med_condition_id  10000 non-null  int64  
 5   medication_id     10000 non-null  int64  
 6   los               10000 non-null  int64  
 7   doctor            10000 non-null  object 
 8   hospital          10000 non-null  object 
 9   billing_amt       10000 non-null  float64
 10  room_number       10000 non-null  int64  
 11  test_results      10000 non-null  object 
dtypes: float64(1), int64(8), object(3)
memory usage: 937.6+ KB


### We will take our transformations and load onto a Google Devian 11 Instance to create a pipeline with Mage VM to Google Cloud Storage. We can perform analysis and visualization with BigQuery and Looker Studio in Google Cloud Storage.
